<div class="alert alert-info">
Привет, Алесандра! Меня зовут Светлана Чих и я буду проверять твой проект. Моя основная цель — не указать на совершенные тобою ошибки, а поделиться своим опытом и помочь тебе. Предлагаю общаться на «ты». Но если это не удобно - дай знать, и мы перейдем на «вы».

<div class="alert alert-success">
<b>👍 Успех:</b> Зелёным цветом отмечены удачные и элегантные решения, на которые можно опираться в будущих проектах.
</div>
<div class="alert alert-warning">
<b>🤔 Рекомендация:</b> Жёлтым цветом выделено то, что в следующий раз можно сделать по-другому. Ты можешь учесть эти комментарии при выполнении будущих заданий или доработать проект сейчас (однако это не обязательно).
</div>
<div class="alert alert-danger">
<b>😔 Необходимо исправить:</b> Красным цветом выделены комментарии, без исправления которых, я не смогу принять проект :(
</div>
<div class="alert alert-info">
<b>👂 Совет:</b> Какие-то дополнительные материалы
</div>
Давай работать над проектом в диалоге: если ты что-то меняешь в проекте по моим рекомендациям — пиши об этом.
Мне будет легче отследить изменения, если ты выделишь свои комментарии:
<div class="alert alert-info"> <b>🎓 Комментарий студента:</b> Например, вот так.</div>
Пожалуйста, не перемещай, не изменяй и не удаляй мои комментарии. Всё это поможет выполнить повторную проверку твоего проекта быстрее.
 </div>

# Описание проекта "Телекоммуникации"

Оператор связи «ТелеДом» хочет бороться с оттоком клиентов. Для этого его сотрудники начнут предлагать промокоды и специальные условия всем, кто планирует отказаться от услуг связи. Чтобы заранее находить таких пользователей, «ТелеДому» нужна модель, которая будет предсказывать, разорвёт ли абонент договор. Команда оператора собрала персональные данные о некоторых клиентах, информацию об их тарифах и услугах. Ваша задача — обучить на этих данных модель для прогноза оттока клиентов.

# План работ

**1)** Загрузить данные и выполнить их первичный осмотр.

**2)** Выполнить исследовательский анализ каждого датафрейма и при необходимости выполнить предобработку. Сделать выводы об имеющихся признаках: понадобятся ли они для обучения моделей.

**3)** Объединить выбранные признаки в один датафрейм по ключу Customer ID, который содержится во всех таблицах.

**4)** Выполнить исследовательский анализ объединённого датафрейма, визуализировать распределения признаков, при необходимости выполнить предобработку. Провести корреляционный анализ. 

**5)** Выполнить подготовку данных для обучения модели. Разделить данные на две выборки, при масштабировании и кодировании учитывать особенности данных и моделей.

**6)** Обучить как минимум две модели. Хотя бы для одной из них подбрать как минимум два гиперпараметра.

**7)** Выбрать лучшую модель и проверить её качество на тестовой выборке.

**8)** Сделать общий вывод о проделанной работе: опишите основные этапы работы, полученные результаты и дать рекомендации для бизнеса.

**Импорты**

In [1]:
!pip install phik -q

In [2]:
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import phik
import scipy.stats as st
import seaborn as sns
import random
import time

RANDOM_STATE = 80724
TEST_SIZE = 0.25

from catboost import CatBoostClassifier
from dateutil.parser import parse as du_parse
from dateutil.relativedelta import relativedelta
from phik import resources, report
from sklearn.datasets import make_classification


from sklearn.linear_model import LogisticRegression                                  
from sklearn.metrics import (
                            roc_auc_score,
                            roc_curve,
                            confusion_matrix
                            )
from sklearn.model_selection import (
                                     TimeSeriesSplit,
                                     train_test_split,
                                     RandomizedSearchCV
                                     )
from sklearn.preprocessing import (StandardScaler, OneHotEncoder)
                                  


ModuleNotFoundError: No module named 'catboost'

<div class="alert alert-success">
<b>👍 Успех:</b> Импортированы нужные библиотеки, определены константы!
</div>

## Шаг. Загрузка данных

***Загрузка таблицы с информацией о договоре.***

In [ ]:
data_contract = pd.read_csv('/datasets/contract_new.csv')

In [ ]:
data_contract.head()

In [ ]:
data_contract.info()

В таблице с договорами 7043 строк, при первичном осмотре пропусков нет, типы данных не везде указаны корректно:

- В столбце с датой начала действия договора BeginDate должен быть тип дата-время (не object);
- В столбце с датой окончания действия договора тип данных object. Формат данных нужно поменять на DateTime, предварительно заменив строки со значениями No на пропуски;
- Столбец 'PaperlessBilling' только 2 значения: Yes и No, поэтому из типа object его нужно перевести в Boollean;
- В столбце MonthlyCharges тип данных float указан верно;
- В столбце TotalCharges тип данных нужно поменять с object на float64, такой же как в столбце MonthlyCharges.

<div class="alert alert-success">
<b>👍 Успех:</b> Все верно!
</div>

***Загрузка таблицы с персональными данными клиентов.***

In [ ]:
data_personal = pd.read_csv('/datasets/personal_new.csv')

In [ ]:
data_personal.head()

In [ ]:
data_personal.info()

В таблице 7043 строк без пропусков. Типы данных указаны не корректно в следующих столбцах:

- SeniorCitizen (является ли клиента пенсионером). Тип нужн перевести в Booleean, здесь возможны только два варианта: Yes (1) или No (0);
- Partner (есть ли у клиента супруг/супруга), Dependents (есть ли у клиента дети). Аналогично предыдущему, возможны только два варианта: Yes (1) или No (0). Нужно перевести в Bollean.

<div class="alert alert-success">
<b>👍 Успех:</b> Все верно!
</div>

***Загрузка таблицы c  информацией об интернет-услугах.***

In [ ]:
data_internet = pd.read_csv('/datasets/internet_new.csv')

In [ ]:
data_internet.head()

In [ ]:
data_internet.info()

В таблице с информацией об интернет-услугах меньше строк, чем в предыдущих, на 21,7% (на 1526 строк). Это связано с тем, что не всем клиентам, с которыми заключены договоры, на данный момент предоставляются интернет-услуги.
Во всех столбцах, кроме InternetService, тип данных нужно сменить с object на boollean.

<div class="alert alert-success">
<b>👍 Успех:</b> Все верно!
</div>

***Загрузка таблицы c информацией об услугах телефонии.***

In [ ]:
data_phone = pd.read_csv('/datasets/phone_new.csv')

In [ ]:
data_phone.head()

In [ ]:
data_phone.info()

В таблице с информацией об услугах телефонии меньше строк, чем первых двух, на 90,7% (на 616 строк). Это значит, что не всем клиентам, с которыми заключены договоры, на данный момент предоставляются услуги телефонии. В столбце MultipleLines тип данных нужно сменить с object на boollean.

<div class="alert alert-success">
<b>👍 Успех:</b> Все верно!
</div>

## Шаг. Исследовательский анализ данных

***Предобработка данных в таблице с информацией о договорах***

Поменяем тип данных в столбце BeginDate на DateTime

In [ ]:
data_contract['BeginDate'] = pd.to_datetime(data_contract['BeginDate'], format = '%Y-%m-%dT%')

Посмотрим уникальные значения в столбце EndDate

In [ ]:
data_contract['EndDate'].value_counts()

Видим, что у 15,6% клиентов (1101) уже истек срок действия договора.

В исходных талицах не было столбца с целевым признаком (ушел клиент или нет). На основании данных столбца EndDate создадим новый столбец с ***целевым признаком Outflow***, в котором будут два значения: 1 (клиент ушел) и 0 (клиент остался). Напишем функцию, которая будет присваивать клиенту категорию в зависимости от значения в столбце EndDate.

In [ ]:
data_contract['Outflow'] = data_contract['EndDate'].apply(lambda x: 0 if x == 'No' else 1)

In [ ]:
data_contract['Outflow'].unique()

In [ ]:
data_contract.head()

<div class="alert alert-success">
<b>👍 Успех:</b> Все верно! Молодец, что проверяшь полученный датасет
</div>

Заменим тип данных на Date Time. Для этого предварительно заменим значения No на пропуски (Nan).

Перед заполнением посмотриим, есть ли пропущенные значения в столбце EndDate.

In [ ]:
data_contract['EndDate'].isna().sum()

In [ ]:
data_contract.loc[data_contract['EndDate'] == 'No', 'EndDate'] = np.nan

Поменяем тип данных в непустых ячейках на DateTime

In [ ]:
data_contract['EndDate'] = pd.to_datetime(data_contract['EndDate'], format = '%Y-%m-%dT%')

In [ ]:
data_contract.tail()


Поменяем тип данных в столбце Outflow на bool.

In [ ]:
data_contract['Outflow'] = data_contract['Outflow'].map({1: True, 0: False})

In [ ]:
data_contract['Outflow'].unique()

In [ ]:
data_contract.info()

Для прогноза может быть полезна информация о сроках, в течение которых клиент пользуется услугами компании. Введем новый признак Duration, в котором будет указан срок обслуживания клиента. В ТЗ сказано, что информация о договрах актуальна на 01.02.2020, поэтому срок действия договора у клиетов, не разорвавших договор, будем считать как разницу между 01.02.2020 и датой заключения договора. У клиентов, разорвавших договор, срок действия, соответственно, это разница между датой окончания и датой заключения.

In [ ]:
def duration_count(row):
    actual_date = pd.to_datetime('2020.02.01')
    if pd.isna(row['EndDate']):
        return (actual_date - row['BeginDate']).days
    else:
        return (row['EndDate'] - row['BeginDate']).days
data_contract['Duration'] = data_contract.apply(duration_count, axis = 1)

In [ ]:
data_contract.head()

<div class="alert alert-success">
<b>👍 Успех:</b> Все верно! Создан новый признак!
</div>

Посмотрим уникальные значения в столбце Type.

In [ ]:
data_contract['Type'].unique()

Признак Type (Тип оплаты) для анализа и прогноза информативен, т к клиенты с типом оплаты на год-два вперед явно более стабильные, чем те, кто платят по месяцам.

<div class="alert alert-success">
<b>👍 Успех:</b> Все верно!
</div>

Столбец PapelessBilling мне кажется не информативным для прогноза, по-моему, его лучше удалить. Но дождемся результатов корреляционного анализа, будем удалять неинформативные признакие после него.

In [ ]:
data_contract['PaymentMethod'].unique()

Столбец с типом платежа информативен для прогноза, т к автоматические списания характерны для более стабильных клиентов. У клиентов с не автоматичнескими платежами вероятность ухода в том числе по причине забыл оплатить выше.

Поменяем тип данных в столбце TotalCharges на float

In [ ]:
data_contract['TotalCharges'] = pd.to_numeric(data_contract.TotalCharges, errors='coerce')

Проверим данные в data_contract на пропуски. Помним, что в столбце EndDate есть 5942 пропуска, которыми мы заменили значения No, чтобы поменять тип данных на DateTime.

In [ ]:
data_contract.isna().sum()

In [ ]:
display(data_contract.loc[data_contract['TotalCharges'].isna()])

Не возможно корректно заполнить пропуски в TotalCharges. Т к число пропусков составляет всего 0,15% от всего датасета, можно эти строки удалить.

<div class="alert alert-danger">
<s><b>😔 Необходимо исправить:</b> Заполнить пропуски возможно, но и удаление тоже допустимый вариант. Почему возникли эти пропуски?</s>
</div>

Дата заключения контракта в строках с пропусками - это дата среза 2020.02.01, т е это клиенты, которые только в день среза данных заключили договоры. В данном случае будет корректно как заполнить пропуски в столбце TotalCharges нулями, так и удалить эти строки. Таких строк немного  - всего 0,15%, поэтому и так, и так можно. Но т к именно эти клиенты пока что еще не оказывают какого-то существенного влияния на статистику (если они только подписали договор, то вряд ли они планируют уходить не успев воспользоваться услугами). Если предположить, что таких клиентов оказалось много, корректнее было бы их удалить.

<div class="alert alert-info"> <b>Комментарий студента:</b> Исправила.</div>

<div class="alert alert-success">
<b>👍 Успех:</b> Все верно!
</div>

In [ ]:
data_contract = data_contract.loc[data_contract['TotalCharges'] >=0]

In [ ]:
data_contract['TotalCharges'].isna().sum()

Удалили пропуски.

Проверим data_contract на явные дулбикаты.

In [ ]:
data_contract.duplicated().sum()

Явных дубликатов нет. Не вижу смысла проверять на неявные, т к в данном случае вполне может быть много клиентов с одинаковыми параметрами (даты договоров, тип оплаты, тип платежа и пр.)

<div class="alert alert-success">
<b>👍 Успех:</b> Все верно!
</div>

***Исследовательский анализ данных таблицы с информаицей о договорах***

Построим гистограммы распределения длительности контрактов у действующих и ушедших клиентов на одном графике.

In [ ]:
x1_ = list(data_contract[data_contract['EndDate'].notna()]['Duration'])
x2_ = list(data_contract[data_contract['EndDate'].isna()]['Duration'])


colors = ['orange', 'green']
names = ['Ушедшие клиенты', 'Актуальные клиенты']

k = plt.figure()
k.set_figwidth(16)
k.set_figheight(7)
plt.hist([x1_, x2_], bins = 80,
         color = colors, label=names)

plt.legend()
plt.title('Гистограммы распределения сроков договоров')
plt.xlabel('Срок действия договоров')
plt.ylabel('Число клиентов')
plt.show();
          

По гистограмме видно, что долгосрочные клиенты (сроком дейстивя договора более 1800 дней) разрывают договоры ощутимо реже, чем клиенты с меньшим сроком обслуживания. То есть этот признак должен коррелировать с вероятностью ухода клиента.

<div class="alert alert-success">
<b>👍 Успех:</b> Все верно!
</div>

Посмотрим распределение признака TotalCharges опять же в разрезе ушедших/действующих клиентов.

In [ ]:
x1_ = list(data_contract[data_contract['EndDate'].notna()]['TotalCharges'])
x2_ = list(data_contract[data_contract['EndDate'].isna()]['TotalCharges'])


colors = ['red', 'blue']
names = ['Ушедшие клиенты', 'Актуальные клиенты']

k = plt.figure()
k.set_figwidth(16)
k.set_figheight(7)
plt.hist([x1_, x2_], bins = 80,
         color = colors, label=names)

plt.legend()
plt.title('Гистограммы распределения общих расходов клиентов')
plt.xlabel('Общие расходы')
plt.ylabel('Число клиентов')
plt.show();
          

Число ушедших клиентов с общими расходами от 6000 стремится к нулю. То есть вероятность ухода клиентов с расходами от 6000 тоже приближается к нулю. Ушедших клиентов с общими расходами 4000-6000 больше, с расходами менее 4000 ощутимо больше. Явно видна корреляция между признаком TotalCharges и вероятностью ухода.

<div class="alert alert-success">
<b>👍 Успех:</b> Все верно!
</div>

Аналогичным образом роанализируем признак MonthlyCharges. По этому признаку можно оценить, сколько услуг предоставляется клиенту. Чем больше сумма - тем больше пакет услуг. 

In [ ]:
x1_ = list(data_contract[data_contract['EndDate'].notna()]['MonthlyCharges'])
x2_ = list(data_contract[data_contract['EndDate'].isna()]['MonthlyCharges'])


colors = ['purple', 'green']
names = ['Ушедшие клиенты', 'Актуальные клиенты']

k = plt.figure()
k.set_figwidth(16)
k.set_figheight(7)
plt.hist([x1_, x2_], bins = 80,
         color = colors, label=names)

plt.legend()
plt.title('Гистограммы распределения расходов за последний месяц')
plt.xlabel('Расходы за последний месяц')
plt.ylabel('Число клиентов')
plt.show();

По этой гистограмме сложно оценить распределение расходов за последний месяц у ушедших клиентов, хорошо видно только распределение расходов у действующих клиентов. Видно, что значительная часть клиентов "сидит" на минимальном пакете услуг стоимостью 20 ден.ед. в месяц.

<div class="alert alert-success">
<b>👍 Успех:</b> Все верно!
</div>

Построим гистограмму распределения расходов за месяц у ушедших клиентов. Для этого выделим их в отдельную таблицу.

In [ ]:
contract_terminate = data_contract[data_contract['EndDate'].notna()]

In [ ]:
contract_terminate['MonthlyCharges'].plot(kind = 'hist', bins = 80, figsize = (20, 5), grid = True, legend = True)
plt.title('Гистограмма распределения расходов за последний месяц у ушедших клиентов', fontsize=15)
plt.xlabel('Расходы за последний месяц у ушедших клиентов', fontsize = 15)
plt.ylabel('Число клиентов', fontsize = 15)
plt.legend(fontsize = 15)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.grid(True)
plt.show()

fig, ax = plt.subplots(figsize = (15,2))
ax = contract_terminate[['MonthlyCharges']].boxplot(vert = False, ax =ax)
ax.set_title('Диаграмма размаха расходов за последний месяц ушедших клиентов')
ax.set_xlabel('Расходы за последний месяц у ушедших клиентов', fontsize = 12)
;

print('Статистика:', contract_terminate['MonthlyCharges'].describe())

Виден пик у клиентов с расходами 20 ден.ед. за месяц, но таких клиентов ощутимо больше, чем всех остальных. Ощутимо больше ушедших клиентов с расходами за последний месяц в интервале 80-110 ден.ед.

<div class="alert alert-success">
<b>👍 Успех:</b> Все верно!
</div>

Построим графики с распределением категориальных признаков: Type, PaperlessBilling,	PaymentMethod.

In [ ]:
features = ['Type', 'PaperlessBilling', 'PaymentMethod']
position = 0
plt.figure(figsize=[18, 6])
plt.subplots_adjust(wspace=0.8, hspace=0.3)
plt.suptitle('Распределение значений категориальных признаков', fontsize=15)
for feature in features:
    position += 1
    plt.subplot(1, 3, position)
    plt.title(f'Распределение признака {feature}', fontsize=15)
    sns.countplot(data = data_contract, x = feature, hue = 'Outflow')
    plt.xlabel(feature, fontsize = 15)
    plt.ylabel('количество', fontsize = 15)
    plt.xticks(fontsize=12)
    plt.yticks(fontsize=12)
    #ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')
    plt.grid(True)

Видим, что число ушедших клиентов с разными типами оплаты примерно одинаково в абсолютном выражении. В относительном выражении (соотношение между оставшимися и ушедшими клиентами) ушедшие клиенты составляют бОльший процент в категориях, которые платят раз в год и раз в два года.

Соотношение между ушедшими и оставшимися клиентами с разными видами чеков не сильно отличаются. У клиентов с электронным чеком соотношение ушедшие/оставшиеся составляет примерно 20%. У клиентов с бумажным чеком это соотношение примерно равно 16%.

Процент ушедших клиентов в категориях  'Bank transfer (automatic)', 'Credit card (automatic)' примерно одинаковый и выше, чем у клиентов с типами оплаты 'Electronic check', 'Mailed check'. То есть вопреки первоначальному предположению клиенты с автоматическим списанием оплаты уходят чаще, чем клиенты с не автоматическим типом оплаты.


<div class="alert alert-success">
<b>👍 Успех:</b> Все верно!
</div>

***Выводы по предобработке и анализу данных таблоицы  data_contract:***

***1)*** Мы поменяли тип данных в столбцах BeginDate и EndDate на DateTime предварительно заменив текстовые значения в EndDate на пропуски.

***2)*** Ввели новый признак длительность договора Duration, определив значения в нем как разницу между EndDate и BeginDate для ушедших клиентов и текущей датой (01.02.2020) и BeginDate для действующих клиентов.

***3)*** Ввели целевой бинарный признак Outflow, если значение равно 1, значит, этот клиент ушел, если 0, клиент остался.

***4)*** Поменяли тип данных в столбце TotalCharges на float и удалили 11 строк (0,15%) с пропусками.

***5)*** Долгосрочные клиенты (сроком действия договора более 1800 дней) разрывают договоры ощутимо реже, чем клиенты с меньшим сроком обслуживания.

***6)*** Видна зависимость между признаком TotalCharges и вероятностью ухода, клиенты с общей суммой расходов более 6000 уходят крайне редко.

***7)*** Среди клиентов с расходами за последний месяц 80-110 ден.ед. ощутимо больше ушедших.

***8)*** В относительном выражении ушедшие клиенты составляют бОльший процент в категориях, которые платят раз в год и раз в два года. Соотношение между ушедшими и оставшимися клиентами с разными видами чеков не сильно отличаются. Клиенты с автоматическим списанием оплаты уходят чаще, чем клиенты с не автоматическим типом оплаты.

<div class="alert alert-success">
<b>👍 Успех:</b> Отличный анализ, молодец!
</div>

***Предобработка данных в таблице с персональными данными***

In [ ]:
data_personal.head()

In [ ]:
data_personal.info()

Переведем тип данных в столбцах SeniorCitizen, Partner, Dependents в boollean.

In [ ]:
data_personal['SeniorCitizen'] = data_personal['SeniorCitizen'].map({1: True, 0: False})

In [ ]:
data_personal['Partner'] = data_personal['Partner'].map({'Yes': True, 'No': False})

In [ ]:
data_personal['Dependents'] = data_personal['Dependents'].map({'Yes': True, 'No': False})

In [ ]:
data_personal.info()

Посмотрим распределения значений по каждому признаку.

In [ ]:
features = ['gender', 'SeniorCitizen', 'Partner', 'Dependents']
position = 0
plt.figure(figsize=[12, 10])
plt.subplots_adjust(wspace=0.5, hspace=0.3)
plt.suptitle('Распределение значений признаков', fontsize=15)
for feature in features:
    position += 1
    plt.subplot(2, 2, position)
    plt.title(f'Распределение признака {feature}', fontsize=12)
    sns.countplot(x = feature, data = data_personal)
    plt.xlabel(feature, fontsize = 15)
    plt.ylabel('количество', fontsize = 15)
    plt.xticks(fontsize=12)
    plt.yticks(fontsize=12)
    plt.grid(True)

Соотношение полов у клиентов 50/50, пенсионеры составляют примерно 15% от общего числа клиентов, по признаку наличия/отсутствия супруга/супруги соотношение клиентов примерно одинаковое, число клиентов, имеющих зависимым лиц (обычно это дети) состаляет менее 30% от общего числа клиентов.

Пропусков в датасете нет, на дубликаты проверять нет смысла, т к в данной таблице совпадающие поля Gender-Senior_citizen-Partner-Dependents это норма.

***Выводы по предобработке и анализу таблицы data_personal:***

***1)*** Заменили тип данных в столбцах с бинарными признаками на bool.

***2)*** Соотношение полов у клиентов 50/50, пенсионеры составляют примерно 15% от общего числа клиентов, по признаку наличия/отсутствия супруга/супруги соотношение клиентов примерно одинаковое, число клиентов, имеющих зависимым лиц (обычно это дети) состаляет менее 30% от общего числа клиентов.

<div class="alert alert-success">
<b>👍 Успех:</b> Все верно!
</div>

***Предобработка и анализ данных таблицы c информацией об интернет-услугах***

In [ ]:
data_internet.head()

In [ ]:
data_internet.info()

В датасете нет пропусков, дубликаты (совпадления по всем полям, кроме индекса) допустимы, т к может быть много клиентов с одинаковыми пакетами услуг.

Все признаки, кроме индекса и InternetServiсe, являются бинарными и по логике надо бы заменить тип данных в этих столбцах с object на bool. Но поскольку в дальнейшем нужно будет объединять все таблицы, то у клиентов, которые не пользуются интернет-услугами, в этих столбцах будут пропуски, которые не корректно будет заполнять нулями. Есть разница между значением "не подключена опция в рамках пакета услуг" (значение False бинарного признака) и значением "отсутствует пакет услуг". После объединения датасетов нужно будет вводить третье значение этих признаков для клиентов, которые не пользуются интернетом, поэтому в бинарный тип в данной ситуации НЕ переводим, оставляем object.

In [ ]:
features = ['InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',\
            'TechSupport', 'StreamingTV', 'StreamingMovies']
position = 0
plt.figure(figsize=[18, 18])
plt.subplots_adjust(wspace=0.3, hspace=0.3)
plt.suptitle('Распределение значений признаков', fontsize=15)
for feature in features:
    position += 1
    plt.subplot(3, 3, position)
    plt.title(f'Распределение признака {feature}', fontsize=12)
    sns.countplot(x = feature, data = data_internet)
    plt.xlabel(feature, fontsize = 12)
    plt.ylabel('количество', fontsize = 12)
    plt.xticks(fontsize=12)
    plt.yticks(fontsize=12)
    plt.grid(True)

У 55% клиентов подключен FiberOptic, у 45% - DSL, у 36% клиентов подключена услуга OnlineSecurity, у 45% подключен OnlineBackup, у 63% подключена DeviceProtection и TechSupport, у половины клиентов подключены StreamingTV и StreamingMovies.

Посмотрим распределение этих признаков у ушедших и оставшихся клиентов на следующем шаге после объединения таблиц.

<div class="alert alert-success">
<b>👍 Успех:</b> Все верно!
</div>

***Предобработка и анализ данных таблицы c информацией об услугах телефонии***

In [ ]:
data_phone.head()

In [ ]:
data_phone.info()

В датасете 6361 строк, то есть 90% клиентов пользуются услугами телефонии. Пропусков нет, на дубликаты не проверяем.

Тип данных у бинарного признака MultipleLines оставляем object по той же причине, что в предыдущей таблице, чтобы корректно заполнить пропуски после объединения таблиц.

In [ ]:
plt.title(f'Распределение признака MultipleLines', fontsize=12)
sns.countplot(x = 'MultipleLines', data = data_phone)
plt.xlabel('Подключение к нескольким линиям', fontsize = 12)
plt.ylabel('Число клиентов', fontsize = 12)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.grid(True)

Примерно 45% клиентов пользуются услугой MultipleLines.

<div class="alert alert-success">
<b>👍 Успех:</b> Все верно!
</div>

## Шаг . Объединение данных

Объединяем таблицы методом merge(). К таблице data_contract присоединяем по очереди все таблицы. Используем "левое" объеднение, т к в исходной таблице data_contract присутствуют все клиенты, и при левом объединении все строки этой таблицы сохранятся.

In [ ]:
data = data_contract.merge(data_personal, how = 'left', on = 'customerID')
data = data.merge(data_internet, how='left', on = 'customerID')
data = data.merge(data_phone, how = 'left', on = 'customerID')

In [ ]:
data.head()

<div class="alert alert-success">
<b>👍 Успех:</b> Объединение прошло верно!
</div>

## Шаг. Исследовательский анализ и предобработка данных объединённого датафрейма

***Предобработка данных объединенного датасета***

Типы данных не трогаем - мы разобрались с ними на предыдущем шаге.

Смотрим на пропуски.

Пропуски в столбце EndDate говорят о том, что в этих строках находятся действующие, а не ушедшие клиенты. Поскольку мы ввели признак длительности контракта, мне кажется,  что признак с датой конца договора можно удалить. Но удаление признаков будем делать после корреляционного анализа.

Пропуски в столбцах с категориальными признаками InternetService, OnlineSecurity, OnlineBackup, DeviceProtection, TechSupport,	StreamingTV, StreamingMovies, MultipleLines заполняем значением 'NotUsed' - это клиенты, которые не пользуются соответствующим типом услуг, поэтому у них отсутствует опция выбора.

In [ ]:
features =  ['InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',\
             'TechSupport', 'StreamingTV', 'StreamingMovies', 'MultipleLines']
for feature in features:
    data.fillna({feature : 'NotUsed'}, inplace = True)

In [ ]:
data.info()

<div class="alert alert-success">
<b>👍 Успех:</b> Все верно!
</div>

***Исследовательский анализ объединенного датасета***

Гистограммы распределения числовых признаков (Duration, TotalCharges, MonthlyCharges) мы смотрели на предыдущем шаге. После объединения нас интересуют распределения ктагориальных признаков из добавленных таблиц в разрезе ушедших/оставшихся клиентов.

In [ ]:
features = ['gender', 'SeniorCitizen', 'Partner', 'Dependents']
position = 0
plt.figure(figsize=[12, 10])
plt.subplots_adjust(wspace=0.5, hspace=0.3)
plt.suptitle('Распределение значений признаков персональных данных', fontsize=15)
for feature in features:
    position += 1
    plt.subplot(2, 2, position)
    plt.title(f'Распределение признака {feature}', fontsize=12)
    sns.countplot(x = feature, data = data, hue = 'Outflow')
    plt.xlabel(feature, fontsize = 15)
    plt.ylabel('количество', fontsize = 15)
    plt.xticks(fontsize=12)
    plt.yticks(fontsize=12)
    plt.grid(True)

Распределение числа клиентов с определенными признаками в абсолютном выражении мне кажется не очень информативным, лучше смотреть, какой процент ушедших клиентов обладал определенными признаками и пользовался определенными услугами.

<div class="alert alert-success">
<b>👍 Успех:</b> Все верно!
</div>

Построим графики с процентами клиентов, обладающими тем или иным признаком.

In [ ]:
outflow_percentage = data.groupby(['Outflow', 'gender']).size() / data.groupby('Outflow').size() * 100
outflow_percentage = outflow_percentage.reset_index(name='percentage')
sns.barplot(data = outflow_percentage, x = 'gender', y = 'percentage', hue = 'Outflow')
plt.title('Процентное соотношение gender')
plt.xlabel('gender')
plt.ylabel('Проценты')
plt.legend(title = 'Outflow')
plt.show()

Соотношение полов у ушедших и оставшихся клиентов - 50/50.

<div class="alert alert-success">
<b>👍 Успех:</b> Все верно!
</div>

In [ ]:
outflow_percentage = data.groupby(['Outflow', 'SeniorCitizen']).size() / data.groupby('Outflow').size() * 100
outflow_percentage = outflow_percentage.reset_index(name='percentage')
sns.barplot(data = outflow_percentage, x = 'SeniorCitizen', y = 'percentage', hue = 'Outflow')
plt.title('Процентное соотношение SeniorCitizen')
plt.xlabel('SeniorCitizen')
plt.ylabel('Проценты')
plt.legend(title = 'Outflow')
plt.show()

20% ушедших клиентов - пенсионеры. 

<div class="alert alert-success">
<b>👍 Успех:</b> Все верно!
</div>

In [ ]:
outflow_percentage = data.groupby(['Outflow', 'Partner']).size() / data.groupby('Outflow').size() * 100
outflow_percentage = outflow_percentage.reset_index(name='percentage')
sns.barplot(data = outflow_percentage, x = 'Partner', y = 'percentage', hue = 'Outflow')
plt.title('Процентное соотношение Partner')
plt.xlabel('Partner')
plt.ylabel('Проценты')
plt.legend(title = 'Outflow')
plt.show()

In [ ]:
outflow_percentage = data.groupby(['Outflow', 'Dependents']).size() / data.groupby('Outflow').size() * 100
outflow_percentage = outflow_percentage.reset_index(name='percentage')
sns.barplot(data = outflow_percentage, x = 'Dependents', y = 'percentage', hue = 'Outflow')
plt.title('Процентное соотношение Dependents')
plt.xlabel('Dependents')
plt.ylabel('Проценты')
plt.legend(title = 'Outflow')
plt.show()

Более 60% ушедших клиентов не имеют супруга/супругу и зависимых лиц.

<div class="alert alert-info"> <b>Комментарий студента:</b> Хотела сделать барплоты с процентами циклом на одной картинке, но что-то пошло не так (смотри ниже), не пойму что. По-моему, загвоздка в строках с outflow_percentage, тут как-то по-другому надо расписать. Что исправить?</div>

<div class="alert alert-warning">
<b>🤔 Рекомендация:</b> По графикам все ОК (они все есть), ты неверно указываешь положения для каждого графика внутри цикла и размер графика
</div>

<div class="alert alert-info"> <b>Комментарий студента:</b> Исправила
</div>

In [ ]:
features = ['gender', 'SeniorCitizen', 'Partner', 'Dependents']
position = 0
plt.figure(figsize=[10, 10])
plt.subplots_adjust(wspace=0.5, hspace=0.3)
plt.suptitle('Распределение значений признаков персональных данных', fontsize=15)
for feature in features:
    position += 1
    outflow_percentage = data.groupby(['Outflow', feature]).size() / data.groupby('Outflow').size() * 100
    outflow_percentage = outflow_percentage.reset_index(name='percentage')
    plt.subplot(2, 2, position)
    sns.barplot(data = outflow_percentage, x = feature, y = 'percentage', hue = 'Outflow')
    plt.title(f'Процентное соотношение {feature}')
    plt.xlabel(f'{feature}')
    plt.ylabel('Проценты')
    plt.legend(title = 'Outflow')

Построим графики распределения процентного соотношения клиентов, разделив их по признаку использования определенных услуг. Посмотрим, какой процент ушедших и оставшихся клиентов пользуется/не пользуется той или иной услугой. После построения графиков сделаем выводы.

In [ ]:
features = ['InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport',\
           'StreamingTV', 'StreamingMovies', 'MultipleLines']
position = 0
plt.figure(figsize=[15, 12])
plt.subplots_adjust(wspace=0.5, hspace=0.3)
plt.suptitle('Распределение значений признаков персональных данных', fontsize=15)
for feature in features:
    position += 1
    outflow_percentage = data.groupby(['Outflow', feature]).size() / data.groupby('Outflow').size() * 100
    outflow_percentage = outflow_percentage.reset_index(name='percentage')
    plt.subplot(3, 3, position)
    sns.barplot(data = outflow_percentage, x = feature, y = 'percentage', hue = 'Outflow')
    plt.title(f'Процентное соотношение {feature}')
    plt.xlabel(f'{feature}')
    plt.ylabel('Проценты')
    plt.grid(True)
    plt.legend(title = 'Outflow')

***Выводы по исследовательскому анализу объединенного датасета:***

Судя по графикам более 50% ушедших клиентов использовали тип связи FiberOptic, почти столько же использовали облачное хранилище для резервного копирования данных, пользовались услугами StreamingTV и StreamingMovies и MultipleLines, в то же время такое же число ушедших (~50%) НЕ пользовались блокировкой опасных сайтов, антивирусом и выделенной линией технической поддержки. 
Более 60% ушедших клиентов не имели супруга/супругу и зависимых лиц.

Можно сделать вывод, что отсутствие услуг блокирвки опасных сайтов, антивируса и выделенной линии техподдержки может быть фактором риска ухода клиента, т к у половины ушедших клиентов не было этих услуг.

<div class="alert alert-success">
<b>👍 Успех:</b> Все верно!
</div>

***Корреляционный анализ***

In [ ]:
data.columns

Поскольку бОльшая часть признаков, включая целевой, являются категориальными или бинарными, будем использовать матрицу корреляции Phik.

In [ ]:
data_phik = data[['BeginDate', 'EndDate', 'Type', 'PaperlessBilling',
       'PaymentMethod', 'MonthlyCharges', 'TotalCharges', 'Outflow',
       'Duration', 'gender', 'SeniorCitizen', 'Partner', 'Dependents',
       'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
       'TechSupport', 'StreamingTV', 'StreamingMovies', 'MultipleLines']]
corr_matrix_data = data_phik.phik_matrix(interval_cols = ['MonthlyCharges', 'TotalCharges', 'Duration'],
                                                    bins = {'MonthlyCharges':20, 'TotalCharges':40, 'Duration':100}) 


In [ ]:
# Визуализация матрицы корреляции phik
plt.figure(figsize=(16, 16))
sns.heatmap(corr_matrix_data, annot=True, cmap='Greens')
plt.title('Phik матрица корреляции для data')
plt.show(); 

Нужно убрать признаки, которые ни на что не влияют или влияют только на зависимый признак:
- EndDate (ощутимо влияет только на Duration, рассчитанный из него;
- PaperlessBilling. Как и предполагалось в начале, мало информативный признак;
- gender;
- SeniorCitizen;
- Dependents. Имеет заивимость только с Partner. Обычно если есть супруг/супруга, то в этом случае есть дети. Поэтому можно убрать Dependents, имеющий слабую или почти нулевую корреляцию с остальными.
Пока просто не будем включать эти признаки в таблицу для расчета матрицы.

Еще предлагаю убрать сильно коррелирующие между собой услуги из интернет-пакета и заменить их новым признаком "Клиент пользуется интернетом или нет". Введем новый признак.

In [ ]:
data['InternetServ'] = data['InternetService'].apply(lambda x: 0 if x == 'NotUsed' else 1)
                                             

In [ ]:
data['InternetServ'] = data['InternetServ'].map({1: True, 0: False})

In [ ]:
data_new_phik = data[['BeginDate', 'Type',\
       'PaymentMethod', 'MonthlyCharges', 'TotalCharges', 'Outflow',\
       'Duration', 'gender', 'SeniorCitizen', 'Partner', 'Dependents', 'InternetService',\
       'InternetServ', 'MultipleLines']]
corr_matrix_data_new = data_new_phik.phik_matrix(interval_cols = ['MonthlyCharges', 'TotalCharges', 'Duration'],
                                                    bins = {'MonthlyCharges':20, 'TotalCharges':40, 'Duration':100}) 

In [ ]:
plt.figure(figsize=(12, 10))
sns.heatmap(corr_matrix_data_new, annot=True, cmap='Greens')
plt.title('Phik матрица корреляции для data')
plt.show(); 

Уберем из таблицы признак BeginDate, имеющий 100% корреляцию с Duration.

Показатели типа подключения InternetService и новый признак InternetServ (пользуется клиент интернетом или нет) имеют 96% и 100% корреляцию с показателем MonthlyCharge и 100% корреляцию друг с другом. Оставим только один (созданный) InternetServ (пользуется клиент интернетом или нет). Создадим новый признак средних расходов за месяц AvrgCharges, рассчитанный как отношение между TotalCharges и числом месяцев, в течение которых клиент обслуживается в компании. 

Создадим новый признак, для этого посчитаем число месяцев, в течение которых действует договор.

Создадим функцию, считающую число месяцев и создадим с помощью нее новый столбец (временный), данные из которого будем использовать для расчета среднемесячных расходов клиентов.

In [ ]:
def duration_month_count(row):
    actual_date = pd.to_datetime('2020.02.01')
    if pd.isna(row['EndDate']):
        d1 = du_parse(str(row['BeginDate']))
        d2 = du_parse(str(actual_date))
        delta = relativedelta(d2, d1)
        return delta.months
    else:
        d1 = du_parse(str(row['BeginDate']))
        d2 = du_parse(str(row['EndDate']))
        delta = relativedelta(d2, d1)
        return delta.months
data['Duration_months'] = data.apply(duration_month_count, axis = 1)

Создадим функцию, которая считает среднемесячные расходы клиентов.

In [ ]:
def avg_charges_count(row):
    if row['Duration_months'] != 0:
        avg_charges = row['TotalCharges']/row['Duration_months']
        return round(avg_charges, 2)
    else:
        avg_charges = row['MonthlyCharges']
        return round(avg_charges, 2)

In [ ]:
data['AvrgCharges'] = data.apply(avg_charges_count, axis = 1)

In [ ]:
data.head()

Включим в таблицу для расчета матрицы новый признак AvrgCharges, а MonthlyCharges уберем. Исключим также InternetService, оставим InternetServ (пользуется клиент интернетом или нет).

In [ ]:
data_new_phik = data[['Type', 'PaymentMethod', 'AvrgCharges', 'TotalCharges', 'Outflow',\
       'Duration', 'SeniorCitizen', 'Partner', 'Dependents', 'InternetServ', 'MultipleLines']]
corr_matrix_data_new = data_new_phik.phik_matrix(interval_cols = ['AvrgCharges', 'TotalCharges', 'Duration'],
                                                    bins = {'AvrgCharges':20, 'TotalCharges':40, 'Duration':100}) 

In [ ]:
plt.figure(figsize=(12, 10))
sns.heatmap(corr_matrix_data_new, annot=True, cmap='Greens')
plt.title('Phik матрица корреляции для data')
plt.show(); 

Уберем неинформативные и сильно коррелирующие с другими признаками столбцы.

In [ ]:
data_final = data.drop(['customerID', 'BeginDate', 'EndDate', 'PaperlessBilling', 'MonthlyCharges', 'InternetService', 'OnlineSecurity', 'OnlineBackup',\
  'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'gender', 'Duration_months'], axis=1)

In [ ]:
data_final.info()

Как видим, показатель среднемесячных расходов AvrgCharges не имеет сильной корреляции с другими входными признаками (в отличие от MonthlyCharges), поэтому AvrgCharges оставляем, MonthlyCharges убираем.

**Выводы по корреляционному анализу:** 

***Убираем*** из таблицы признаки, которые не несут информативной нагрузки и которые имеют высокую корреляцию с другими входными признаками:

- 'BeginDate',
- 'EndDate', 
- 'PaperlessBilling', 
- услуги из интернет-пакета: 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies',
- gender,
- MonthlyCharges (признак имеет 100% корреляцию с признаками из data_internet),
- Duration_months (мы использовали этот признак для расчета нового, для анализа и построения модели он лишний, т к имеет высокую корреляцию с Duration, но при этом менее точный, чем Duration.

Мы ввели следующие ***новые признаки***:

- InternetServ - вместо перечня услуг из таблицы data_internet,
- Duration - срок действия договора,
- AvrgCharges - среднемесячные расходы. Этот признак не имеет высокой корреляции с другими входными. И он на мой взгляд более информативен, т к более объективно отражает историю взаимодействия с клиентом. MonthlyCharges отражает сумму платежа только за последний месяц, а клиенты в течение срока действия договора могли много раз менять тарифы, подключать новые и отказываться от старых услуг.

<div class="alert alert-success">
<b>👍 Успех:</b> Отличный корреляционный анализ, молодец!
</div>

## Шаг. Подготовка данных и обучение моделей

Будем обучать две модели:

 - Логистическая регрессия,
 - Градиентный бустинг CatBoost.
 
 

В начале подготовим признаки и обучим модель логистической регрессии, т к для обучения модели бустинга нужно будет переводить категориальные признаки в числовой тип.

Разделим датасет на ренировочную и тестовую выборки.

In [ ]:
features = data_final.drop(columns = 'Outflow')
target = data_final['Outflow']

features_train, features_test, target_train, target_test = \
            train_test_split(features, target, test_size = TEST_SIZE, random_state = RANDOM_STATE)

print(f'Размер тренировочной выборки - {target_train.shape[0]}')
print(f'Размер тестовой выборки - {target_test.shape[0]}')

Для кодирования категориальных признаков будем использовать OHE кодировщик, т к все категориальные признаки бинарные, ранговых нет. Для числовых будем использовать StandardScaler.

In [ ]:
cat_col_names = ['Type', 'PaymentMethod', 'SeniorCitizen', 'Partner', 'Dependents', 'MultipleLines', 'InternetServ']
num_col_names = ['TotalCharges', 'Duration', 'AvrgCharges']

Кодируем категориальные признаки.

In [ ]:
# создадим переменную с экземпляром класса OneHotEncoder()
encoder_ohe = OneHotEncoder(drop = 'first', sparse_output = False)

# обучим и трансформируем категориальные признаки из тренировочной выборки с помощью  OneHotEncoder
features_train_ohe = encoder_ohe.fit_transform(features_train[cat_col_names])

# выполним преобразование категориальных признаков в тестовой выборке
features_test_ohe = encoder_ohe.transform(features_test[cat_col_names])

# получим от кодировщика список новых столбцов и сохраним в новой переменной 
encoder_col_names = encoder_ohe.get_feature_names_out()

# создаём тренировочный и тестовый датафреймы из закодированных данных

features_train_ohe = pd.DataFrame(features_train_ohe, columns = encoder_col_names)
features_test_ohe = pd.DataFrame(features_test_ohe, columns = encoder_col_names)

Масштабируем количественные признаки.

In [ ]:
# создадим переменную с экземпляром класса StandardScaler()
scaler = StandardScaler()

# обучим и трансформируем количественные признаки из тренировочной выборки 
features_train_scaled = scaler.fit_transform(features_train[num_col_names])

# выполним преобразование количественных признаков в тестовой выборке
features_test_scaled = scaler.transform(features_test[num_col_names])

# создаём тренировочный и тестовый датафреймы из отмасштабированных данных
features_train_scaled = pd.DataFrame(features_train_scaled, columns = num_col_names)
features_test_scaled = pd.DataFrame(features_test_scaled, columns = num_col_names)

Объединим кодированные и масштабированные признаки в тренировочный и тестовый датафреймы.

In [ ]:
features_train = pd.concat([features_train_ohe.reset_index(), features_train_scaled.reset_index()], axis=1)

features_test = pd.concat([features_test_ohe.reset_index(), features_test_scaled.reset_index()], axis=1)

In [ ]:
target_train.unique()

<div class="alert alert-success">
<b>👍 Успех:</b> Все верно!
</div>

In [ ]:
start_time = time.time()
param_logr = {'C': [0.01, 0.1, 1, 10, 100],
                'class_weight': [None, 'balanced']
               }

model_logr = LogisticRegression()

search_logr = RandomizedSearchCV(model_logr, param_logr, cv = 3, scoring = 'roc_auc', verbose = False)
search_logr.fit(features_train, target_train)
end_time = time.time()

training_time_logr = end_time - start_time
training_time_logr = round(training_time_logr, 2)

print('Лучшие параметры:', search_logr.best_params_)
print('Лучший результат:', search_logr.best_score_)
print('Время обучения модели логистической регрессии: ', training_time_logr)

<div class="alert alert-danger">
<s><b>😔 Необходимо исправить:</b> Визуализировать ROC кривую нужно только для лучшей модели на тестовых данных, считать predict_proba на тех же данных, на которых обучалась модель не стоит, у тебя уже выведена метрика best_score_</s>
</div>

<div class="alert alert-info"> <b>Комментарий студента:</b> Здесь убрала, ниже сделала для лучше модели</div>

<div class="alert alert-success">
<b>👍 Успех:</b> Все верно!
</div>

***Модель Catboost***

Для обучения модели бустинга заменим тип даных на числовой.

In [ ]:
features =  ['Type', 'PaymentMethod', 'SeniorCitizen', 'Partner', 'Dependents', 'MultipleLines', 'InternetServ']
for feature in features:
    data_final[feature] = data_final[feature].astype('category').cat.codes

Разделим на тренировочную и тестовую.

In [ ]:
features = data_final.drop(columns = 'Outflow')
target = data_final['Outflow']

features_train, features_test, target_train, target_test = \
            train_test_split(features, target, test_size = TEST_SIZE, random_state = RANDOM_STATE)

print(f'Размер тренировочной выборки - {target_train.shape[0]}')
print(f'Размер тестовой выборки - {target_test.shape[0]}')

In [ ]:
start_time = time.time()
param_ctbst = {'iterations': [10, 50, 100, 200],
                'learning_rate': [0.01, 0.1, 0.15],
                'depth': [3, 4, 5, 6]
               }

model_ctbst = CatBoostClassifier(random_state = RANDOM_STATE)

search_ctbst = RandomizedSearchCV(model_ctbst, param_ctbst, cv = 3, scoring = 'roc_auc', verbose = False)
search_ctbst.fit(features_train, target_train)
end_time = time.time()
training_time_catboost = end_time - start_time
training_time_catboost = round(training_time_catboost, 2)

print('Лучшие параметры:', search_ctbst.best_params_)
print('Лучший результат:', search_ctbst.best_score_)
print('Время обучения модели Catboost:', training_time_catboost)

Сделаем предсказания на тренировчоной выборке с помощью модели CatBoost.

<div class="alert alert-danger">
    <s><b>😔 Необходимо исправить:</b> Здесь тоже не нужно строить ROC кривую</s>
</div>

<div class="alert alert-info"> <b>Комментарий студента:</b> убрала</div>

<div class="alert alert-success">
<b>👍 Успех:</b> Все верно!
</div>

## Шаг. Выбор лучшей модели

Выведем таблицу с результатами по каждой из моделей.

In [ ]:
results = pd.DataFrame({'Модель': ['LogRegression', 'CatBoost'],
                        'Время обучения, секунд': [training_time_logr, training_time_catboost],
                        'Параметры лучших моделей': [search_logr.best_params_, search_ctbst.best_params_],
                        'ROC_AUC': [search_logr.best_score_, search_ctbst.best_score_]})

# Вывод результатов
results

Модель CatBoost показала меньшее, чем LogReg, время обучения  - 20,4 секунды, и намного более высокую метрику ROC_AUC = 0.89, удовлетворяющую требованиям заказчика (не менее 0.85).

<div class="alert alert-danger">
<s><b>😔 Необходимо исправить:</b> Метрика не достигнута, для лучшей модели нужно визуализировать ROC кривую, важность признаков и матрицу ошибок</s>
</div>

<div class="alert alert-info"> <b>Комментарий студента:</b> Перед отправкой 2 раза делала перезапуск всех ячекк - получалась метрика 0.89. </div>

<div class="alert alert-success">
<b>👍 Успех:</b> Все верно! Метрика получилась выше пороговой
</div>

Проверим качество лучшей модели на тестовой выборке.

In [ ]:
predict_ctbst_test = search_ctbst.best_estimator_.predict_proba(features_test)[:, 1]
roc_auc = roc_auc_score(target_test, predict_ctbst_test)
print('Метрика ROC_AUC лучшей модели: ', roc_auc)

На тестовой выборке модель показала еще более высокую метрику почти 92%.

Проанализируем важность признаков модели CatBoost.

In [ ]:
coefs = pd.DataFrame(search_ctbst.best_estimator_.feature_importances_, index = features_test.columns, columns=['coefs'])
coefs = coefs.sort_values(by = 'coefs', ascending = False)
coefs.iloc[:10]

In [ ]:
coefs.nlargest(10, columns = ['coefs']).plot(kind='barh', title = 'Важность признаков', xlabel = 'Признаки')

Признаки, вносящие максимальный вклад в результаты прогноза - это введенные нами признаки Duration (длительность контракта) и сроеднемесячные расходы (AvrgCharges).
Признаки, не влияющие на результат прогноза - наличие детей, факт наличия у клиента пакета интернет-услуг и возраст (пенсионер или нет). Фактически не влияет на результат ипредсказания наличие у клиента супруга/супруги.

<div class="alert alert-success">
<b>👍 Успех:</b> Все верно!
</div>

Визуализируем ROC_AUC кривую для лучшей модели.


In [ ]:
predict_ctbst_test = search_ctbst.best_estimator_.predict_proba(features_test)[:, 1]
roc_auc = roc_auc_score(target_test, predict_ctbst_test)
print('Метрика ROC_AUC лучшей модели: ', roc_auc)

In [ ]:
fpr, tpr, thresholds = roc_curve(target_test, predict_ctbst_test)  
plt.plot(fpr, tpr)     
plt.title("График зависимости FPR от TPR")
plt.xlabel('False Positive Rate (FPR)')
plt.ylabel('True Positive Rate (TPR)')
plt.grid(True)
plt.show(); 

Площадь фигуры под кривой при визуальной оценке (~23/25 ~0.92) соотвествует полученной на тестовой выборке метрике.

Построим матрицу ошибок, предварительно посчитаем метки классов (до этого считали вероятности меток класса 1).

In [ ]:
predict_class_ctbst = search_ctbst.best_estimator_.predict(features_test)

In [ ]:
target_test.unique()

In [ ]:
predict_class_ctbst = [eval(elt) for elt in predict_class_ctbst]

In [ ]:
cm = confusion_matrix(target_test, predict_class_ctbst)
plt.figure(figsize = (5,5))
plt.title("Матрица ошибок")
ax = sns.heatmap(cm, annot=True, fmt='d', cmap='Blues_r')
plt.xlabel('Predicted label')
plt.ylabel('True label');

***Интерпретация матрицы ошибок:***

***1)*** Модель правильно предсказала 137 "отточных" клиентов из 273 (137+136 всего отточных клиентов), т е правильно предсказана половина из тех клиентов, которые реально собираются уйти. Recall равна 50%.

***2)*** Модель правильно предсказала 1473 клиента, которые не планируют разрывать контракт. Precision = 92% (137/(137+12)).

***3)*** Всего 12 клиентов были ошибочно классифицированы как False Positive, т е ошибок первого рода всего 12, компания  понесет минимум лишних расходов на удержание клиентов, которые не планировали уходить.

***4)*** 136 клиентов  были классифицированы как False Negative (ошибок 2 рода 136). То есть 136 клиентов, которые на самом деле планирует разорвать контракт (половина всех "отточных" клиентов), будет ошибочно классифицированы моделью как нормальные (не планирующие уходить).


<div class="alert alert-success">
<b>👍 Успех:</b> Все верно!
</div>

## Шаг. Общий вывод и рекомендации заказчику

***1)*** В ходе работы мы привели данные к нужному типу, обработали пропуски.

***2)*** Ввели новый признак длительности договора Duration и целевой признак Outflow.

***3)*** Объединили все таблицы с данными и провели исследовательский анализ получившегося датасета. Сделали предположение, что отсутствие услуг блокировки опасных сайтов, антивируса и выделенной линии техподдержки может быть фактором риска ухода клиента, т к у половины ушедших клиентов не было этих услуг. 

***4)*** Провели корреляционный анализ, по итогам которого убрали из таблицы признаки, не несущие информативной нагрузки или имеющие высокую корреляцию с другими входными признаками. Ввели новые признаки: InternetServ (пользуется клиент интернетом или нет) и AvrgCharges (среднемесячные расходы).

***5)*** Обучили две модели для предсказания оттока клиентов. Для проверки качества моделей использовали метрику roc_auc. Лучшее качество показала модель CatBoost, ROC_AUC полученной модели на тренировочной выборке - 0.89, на тестовой выборке - 0.92. ***Поэтому можем рекомендовать модель CatBoost для предсказания вероятности ухода клиентов.***

***6)*** Самыми важными признаками, влияющими на предсказания модели, стали введенные нами признаки длительности контракта (Duration) и среднемесячных расходов (AvrgCharges). 

Признаки, не влияющие на качество модели (а значит, и на вероятность ухода клиента):

- наличие у клиента семьи (супруга/супруги и/или детей), 
- возраст клиента (является пенсионером или нет), 
- наличие у клиента пакета инетрнет-услуг.

***7)*** Получили очень высокую метрику accuracy 99,2%. Такая доля остающихся клиентов будет классифицирована моделью правильно, компания понесет мимимум лишних расходов на удержание и так стабильных клиентов. Метрика recall получилась 50%, то есть половина клиентов, планирующих разорвать контракт, будут классифицированы как уходящие.

<div class="alert alert-success">
<b>👍 Успех:</b> Молодец, финальный проект завершен! Сама работа получилась хорошей и структурированной, были предобработаны и проанализированы данные, выбран целевой признак, рассчеты подкреплены визуализацией, это очень важная часть работы, которая облегчает анализ и позволяет полнее представлять происходящее в данных. Построено и обучено несколько моделей, все они оценены и выбрана лучшая. Цель работы достигнута, получена модель хорошо предсказывающая уход клиентов. Не забывай о том, что все пункты очень важны и каждому стоит уделять максимум внимания. <p>
С опытом становится значительно легче, но опыт это не только повторение однажды изученного, но и постоянное развитие, тем более, что ты выбрала очень динамично развивающуюся область. <p>
В будущей профессии тебе точно пригодиться умение системно подходить к решению аналитических задач, здесь рекомендую изучить ТРИЗ и системный анализ, из литературы можно почитать Теоретический минимум по Big Data — Су Кеннет и Ын Анналин, Практическая статистика для специалистов Data Science — Брюс П. и Брюс Э., Real-World Machine Learning — Henric Brink, Joseph Мark, W. Richards Fetherolf, Прикладное машинное обучение с помощью Scikit-Learn и TensorFlow — Жерон Орельен.<p>
Есть интересные сообщества (например https://vk.com/mashinnoe_obuchenie_ai_big_data) и конечно же https://habr.com/ru/all/<p>

Дополнительно предлагаю посмотреть:
- Книга от ШАД: https://academy.yandex.ru/handbook/ml

- Открытый курс машинного обучения: https://habr.com/ru/company/ods/blog/322626/

 Удачи тебе и профессионального роста!
</div>